In [18]:
%matplotlib inline

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [16]:
from __future__ import division

import sys
import time
import logging

import numpy as np
from scipy.io import loadmat
from os import path

# change this to the path of mushu if you don't have it in your
# PYTHONPATH already
sys.path.append('../../mushu')
sys.path.append('../')

import libmushu
from libmushu.driver.replayamp import ReplayAmp
from wyrm.types import RingBuffer
import wyrm.processing as proc
from wyrm import io
from wyrm.types import Data, BlockBuffer, RingBuffer

#Classifiers
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
import matplotlib
import matplotlib.pyplot as plt

In [2]:
logging.basicConfig(format='%(relativeCreated)10.0f %(threadName)-10s %(name)-10s %(levelname)8s %(message)s', level=logging.NOTSET)
logger = logging.getLogger(__name__)

In [3]:
# replay the experiment in real time?
REALTIME = False


TRAIN_DATA = '../BCI_Comp_III_Wads_2004/data/Subject_A_Train.mat'
TEST_DATA = '../BCI_Comp_III_Wads_2004/data/Subject_A_Test.mat'

CHANNEL_DATA = '../BCI_Comp_III_Wads_2004/data/eloc64.txt'

TRUE_LABELS_TEST = "WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU"

STIMULUS_CODE = {
    # cols from left to right
    1 : "agmsy5",
    2 : "bhntz6",
    3 : "ciou17",
    4 : "djpv28",
    5 : "ekqw39",
    6 : "flrx4_",
    # rows from top to bottom
    7 : "abcdef",
    8 : "ghijkl",
    9 : "mnopqr",
    10: "stuvwx",
    11: "yz1234",
    12: "56789_"
}

MARKER_DEF_TRAIN = {'target': ['target'], 'nontarget': ['nontarget']}
MARKER_DEF_TEST = {i : [i] for i in STIMULUS_CODE.values()}

SEG_IVAL = [0, 800]

In [4]:
def load_bci_TESTdata(filename):
    """Load the BCI Competition III Data Set 2.
    This method loads the data set and converts it into Wyrm's ``Data``
    format. Before you use it, you have to download the data set in
    Matlab format and unpack it. The directory with the extracted files
    must contain the ``Subject_*.mat``- and the ``eloc64.txt`` files.
    .. note::
        If you need the true labels of the test sets, you'll have to
        download them separately from
        http://bbci.de/competition/iii/results/index.html#labels
    Parameters
    ----------
    filename : str
        The path to the matlab file to load
    Returns
    -------
    cnt : continuous `Data` object
    Examples
    --------
    >>> dat = load_bcicomp3_ds2('/home/foo/data/Subject_A_Train.mat')
    """
    STIMULUS_CODE = {
        0 : "blankMatrix",
        # cols from left to right
        1 : "agmsy5",
        2 : "bhntz6",
        3 : "ciou17",
        4 : "djpv28",
        5 : "ekqw39",
        6 : "flrx4_",
        # rows from top to bottom
        7 : "abcdef",
        8 : "ghijkl",
        9 : "mnopqr",
        10: "stuvwx",
        11: "yz1234",
        12: "56789_"
        }

    # load the matlab data
    data_mat = loadmat(filename)
    # load the channel names (the same for all datasets
    eloc_file = path.sep.join([path.dirname(filename), 'eloc64.txt'])
    with open(eloc_file) as fh:
        data = fh.read()
    channels = []
    for line in data.splitlines():
        if line:
            chan = line.split()[-1]
            chan = chan.replace('.', '')
            channels.append(chan)
    # fix the channel names, some letters have the wrong capitalization
    for i, s in enumerate(channels):
        s2 = s.upper()
        s2 = s2.replace('Z', 'z')
        s2 = s2.replace('FP', 'Fp')
        channels[i] = s2
    # The signal is recorded with 64 channels, bandpass filtered
    # 0.1-60Hz and digitized at 240Hz. The format is Character Epoch x
    # Samples x Channels
    data = data_mat['Signal'][60:,:,:]
    data = data.astype('double')
#     print('data: ',data.shape)
    # For each sample: 1 if a row/colum was flashed, 0 otherwise
    flashing = data_mat['Flashing'][60:,:]
    flashing = flashing.reshape(-1)
#     print('flashing: ',flashing.shape)
    #flashing = np.flatnonzero((np.diff(a) == 1)) + 1
    ##Creates an array where only the initial intensifications of each series appear
    tmp = []
    for i, _ in enumerate(flashing):
        if i == 0:
            tmp.append(flashing[i])
            continue
        if flashing[i] == flashing[i-1] == 1:
            tmp.append(0)
            continue
        tmp.append(flashing[i])
    flashing = np.array(tmp)
    # For each sample: 0 when no row/colum was intensified,
    # 1..6 for intensified columns, 7..12 for intensified rows
    stimulus_code = data_mat['StimulusCode'][60:,:].reshape(-1)
#     print('stim_code: ', stimulus_code.shape)
    stimulus_code = stimulus_code[flashing == 1]
    # 0 if no row/col was intensified or the intensified did not contain
    # the target character, 1 otherwise
    stimulus_type = data_mat.get('StimulusType', np.array([]))[60:,:].reshape(-1)
#     print('stim_type: ', stimulus_type.shape)
    # The target characters
    target_chars = data_m.get('TargetChar')[0][60:]
#     target_chars = data_mat.get('TargetChar', np.array([])).reshape(-1)
    fs = 240
    data = data.reshape(-1, 64)
    timeaxis = np.linspace(0, data.shape[0] / fs * 1000, data.shape[0], endpoint=False)
    dat = Data(data=data, axes=[timeaxis, channels], names=['time', 'channel'], units=['ms', '#'])
    dat.fs = fs
    # preparing the markers
    target_mask = np.logical_and((flashing == 1), (stimulus_type == 1)) if len(stimulus_type) > 0 else []
    nontarget_mask = np.logical_and((flashing == 1), (stimulus_type == 0)) if len(stimulus_type) > 0 else []
    flashing = (flashing == 1)
    flashing = [[i, 'flashing'] for i in timeaxis[flashing]]
    targets = [[i, 'target'] for i in timeaxis[target_mask]]
    nontargets = [[i, 'nontarget'] for i in timeaxis[nontarget_mask]]
    dat.stimulus_code = stimulus_code[:]
    stim = []
    for i,_ in enumerate(flashing):
        stim.append([flashing[i][0], STIMULUS_CODE[stimulus_code[i]]])
    stimulus_code = stim
    #stimulus_code = zip([t for t, _ in flashing], [STIMULUS_CODE[i] for i in stimulus_code])
    #Raises error "TypeError: '<' not supported between instances of 'tuple' and 'list'" when calling sort() 
    #stimulus_code =[[t for t,_ in flashing], [STIMULUS_CODE[i] for i in stimulus_code]]
    #print(type(stimulus_code), type(flashing), type(targets), type(nontargets))
    markers = flashing[:]
    markers.extend(targets)
    markers.extend(nontargets)
    markers.extend(stimulus_code)
    markers.sort()
    dat.markers = markers[:]
    return dat


In [5]:
def load_bci_TRAINdata(filename):
    """Load the BCI Competition III Data Set 2.
    This method loads the data set and converts it into Wyrm's ``Data``
    format. Before you use it, you have to download the data set in
    Matlab format and unpack it. The directory with the extracted files
    must contain the ``Subject_*.mat``- and the ``eloc64.txt`` files.
    .. note::
        If you need the true labels of the test sets, you'll have to
        download them separately from
        http://bbci.de/competition/iii/results/index.html#labels
    Parameters
    ----------
    filename : str
        The path to the matlab file to load
    Returns
    -------
    cnt : continuous `Data` object
    Examples
    --------
    >>> dat = load_bcicomp3_ds2('/home/foo/data/Subject_A_Train.mat')
    """
    STIMULUS_CODE = {
        0 : "blankMatrix",
        # cols from left to right
        1 : "agmsy5",
        2 : "bhntz6",
        3 : "ciou17",
        4 : "djpv28",
        5 : "ekqw39",
        6 : "flrx4_",
        # rows from top to bottom
        7 : "abcdef",
        8 : "ghijkl",
        9 : "mnopqr",
        10: "stuvwx",
        11: "yz1234",
        12: "56789_"
        }

    # load the matlab data
    data_mat = loadmat(filename)
    # load the channel names (the same for all datasets
    eloc_file = path.sep.join([path.dirname(filename), 'eloc64.txt'])
    with open(eloc_file) as fh:
        data = fh.read()
    channels = []
    for line in data.splitlines():
        if line:
            chan = line.split()[-1]
            chan = chan.replace('.', '')
            channels.append(chan)
    # fix the channel names, some letters have the wrong capitalization
    for i, s in enumerate(channels):
        s2 = s.upper()
        s2 = s2.replace('Z', 'z')
        s2 = s2.replace('FP', 'Fp')
        channels[i] = s2
    # The signal is recorded with 64 channels, bandpass filtered
    # 0.1-60Hz and digitized at 240Hz. The format is Character Epoch x
    # Samples x Channels
    data = data_mat['Signal'][:60,:,:]
    data = data.astype('double')
#     print('data: ',data.shape)
    # For each sample: 1 if a row/colum was flashed, 0 otherwise
    flashing = data_mat['Flashing'][:60,:]
    flashing = flashing.reshape(-1)
#     print('flashing: ',flashing.shape)
    #flashing = np.flatnonzero((np.diff(a) == 1)) + 1
    ##Creates an array where only the initial intensifications of each series appear
    tmp = []
    for i, _ in enumerate(flashing):
        if i == 0:
            tmp.append(flashing[i])
            continue
        if flashing[i] == flashing[i-1] == 1:
            tmp.append(0)
            continue
        tmp.append(flashing[i])
    flashing = np.array(tmp)
    # For each sample: 0 when no row/colum was intensified,
    # 1..6 for intensified columns, 7..12 for intensified rows
    stimulus_code = data_mat['StimulusCode'][:60,:].reshape(-1)
#     print('stim_code: ', stimulus_code.shape)
    stimulus_code = stimulus_code[flashing == 1]
    # 0 if no row/col was intensified or the intensified did not contain
    # the target character, 1 otherwise
    stimulus_type = data_mat.get('StimulusType', np.array([]))[:60,:].reshape(-1)
#     print('stim_type: ', stimulus_type.shape)
    # The target characters
    target_chars = data_m.get('TargetChar')[0][:60]
#     target_chars = data_mat.get('TargetChar', np.array([])).reshape(-1)
    
    fs = 240
    data = data.reshape(-1, 64)
    timeaxis = np.linspace(0, data.shape[0] / fs * 1000, data.shape[0], endpoint=False)
    dat = Data(data=data, axes=[timeaxis, channels], names=['time', 'channel'], units=['ms', '#'])
    dat.fs = fs
    # preparing the markers
    target_mask = np.logical_and((flashing == 1), (stimulus_type == 1)) if len(stimulus_type) > 0 else []
    nontarget_mask = np.logical_and((flashing == 1), (stimulus_type == 0)) if len(stimulus_type) > 0 else []
    flashing = (flashing == 1)
    flashing = [[i, 'flashing'] for i in timeaxis[flashing]]
    targets = [[i, 'target'] for i in timeaxis[target_mask]]
    nontargets = [[i, 'nontarget'] for i in timeaxis[nontarget_mask]]
    dat.stimulus_code = stimulus_code[:]
    stim = []
    for i,_ in enumerate(flashing):
        stim.append([flashing[i][0], STIMULUS_CODE[stimulus_code[i]]])
    stimulus_code = stim
    #stimulus_code = zip([t for t, _ in flashing], [STIMULUS_CODE[i] for i in stimulus_code])
    #Raises error "TypeError: '<' not supported between instances of 'tuple' and 'list'" when calling sort() 
    #stimulus_code =[[t for t,_ in flashing], [STIMULUS_CODE[i] for i in stimulus_code]]
    #print(type(stimulus_code), type(flashing), type(targets), type(nontargets))
    markers = flashing[:]
    markers.extend(targets)
    markers.extend(nontargets)
    markers.extend(stimulus_code)
    markers.sort()
    dat.markers = markers[:]
    return dat


In [6]:
def train(filename):
    cnt = load_bci_TRAINdata(filename)

    fs_n = cnt.fs / 2

    b, a = proc.signal.butter(5, [10 / fs_n], btype='low')
    cnt = proc.lfilter(cnt, b, a)

#     b, a = proc.signal.butter(5, [.4 / fs_n], btype='high')
#     cnt = proc.lfilter(cnt, b, a)

    cnt = proc.subsample(cnt, 20)

    epo = proc.segment_dat(cnt, MARKER_DEF_TRAIN, SEG_IVAL)

    #from wyrm import plot
    #plot.plot_spatio_temporal_r2_values(proc.sort_channels(epo))
    #print JUMPING_MEANS_IVALS
    #plot.plt.show()

    fv = proc.create_feature_vectors(epo)
    X = fv.data
    y = fv.axes[0]
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, train_size = 0.7)

    #     clf = proc.lda_train(fv)
    clf = LinearDiscriminantAnalysis(solver='eigen')
    clf.fit(X,y)
    return clf

In [82]:
def online_experiment(amp, cfy):
    amp_fs = amp.get_sampling_frequency()
    amp_channels = amp.get_channels()

    #buf = BlockBuffer(4)
    rb = RingBuffer(5000)

    fn = amp_fs / 2
    b_low, a_low = proc.signal.butter(5, [10 / fn], btype='low')
#     b_high, a_high = proc.signal.butter(5, [.4 / fn], btype='high')

    zi_low = proc.lfilter_zi(b_low, a_low, len(amp_channels))
#     zi_high = proc.lfilter_zi(b_high, a_high, len(amp_channels))

    amp.start()
    markers_processed = 0
    current_letter_idx = 0
    current_letter = TRUE_LABELS[current_letter_idx].lower()

    letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
    endresult = []

    t0 = time.time()
    data, markers = amp.get_data()
    y_true = np.array([])
    y_pred = np.array([])
    while True:
        t0 = time.time()

        # get fresh data from the amp
        data, markers = amp.get_data()
        if len(data) == 0:
            continue

        # we should rather wait for a specific end-of-experiment marker
        if len(data) == 0:
            break

        # convert to cnt
        cnt = io.convert_mushu_data(data, markers, amp_fs, amp_channels)

        ## enter the block buffer
        #buf.append(cnt)
        #cnt = buf.get()
        #if not cnt:
        #    continue

        # low-pass and subsample
        cnt, zi_low = proc.lfilter(cnt, b_low, a_low, zi=zi_low)
#         cnt, zi_high = proc.lfilter(cnt, b_high, a_high, zi=zi_high)

        cnt = proc.subsample(cnt, 20)

        newsamples = cnt.data.shape[0]

        # enter the ringbuffer
        rb.append(cnt)
        cnt = rb.get()

        # segment
        epo = proc.segment_dat(cnt, MARKER_DEF_TEST, SEG_IVAL, newsamples=newsamples)
        epo_train = proc.segment_dat(cnt, MARKER_DEF_TRAIN, SEG_IVAL, newsamples=newsamples)
        if not epo:
            continue

        fv = proc.create_feature_vectors(epo)
        fv_train = proc.create_feature_vectors(epo_train)
        
        logger.debug('Markers processed: ')
        logger.debug(markers_processed)
#         logger.debug(markers_processed)
        
        lda_out = cfy.predict_proba(fv.data)[:,1]
        y_true = np.append(y_true, fv_train.axes[0])
    
        y_pred = np.append(y_pred, cfy.predict(fv.data))

#         lda_out = proc.lda_apply(fv, cfy)
        markers = [fv.class_names[cls_idx] for cls_idx in fv.axes[0]]
        result = zip(markers, lda_out)
        
        report = metrics.classification_report(y_true, y_pred)
        recall = metrics.recall_score(y_true, y_pred)
        precision = metrics.precision_score(y_true, y_pred)
        accuracy = metrics.accuracy_score(y_true, y_pred)
        f1 = metrics.f1_score(y_true, y_pred)
        print('P300 classification metrics...')
        print('recall: ', recall, '\tprecision: ', precision, '\tf1: ', f1, '\taccuracy: ', accuracy)
        for s, score in result: #letters in one row/column

#             report = metrics.classification_report(y_true, y_pred)
#             print(report)
#             print(s, score)
            if markers_processed == 180: # 12 trials x 15 repetitions = 180
                endresult.append(sorted(letter_prob.items(), key=lambda x: x[1])[-1][0])
                letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
                markers_processed = 0
                current_letter_idx += 1
                current_letter = TRUE_LABELS[current_letter_idx].lower()
            for letter in s:
#                 print('letter', letter, '\tscore: ', score)
                letter_prob[letter] += score
            markers_processed += 1
#         print(letter_prob)
#         logger.debug("".join([i[0] for i in sorted(letter_prob.items(), key=lambda x: x[1], reverse=True)]).replace(current_letter, " %s " % current_letter))

        # calculate the current accuracy
        if len(endresult) > 0:
            logger.debug(TRUE_LABELS)
            logger.debug("".join(endresult))
            acc = np.count_nonzero(np.array(endresult) == np.array(list(TRUE_LABELS.lower()[:len(endresult)]))) / len(endresult)
            print('Character prediction metrics...')
            print("Current accuracy:", acc * 100)
        if len(endresult) == len(TRUE_LABELS):
            break
        #logger.debug("Result: %s" % result)
        print('time',1000 * (time.time() - t0))

    acc = np.count_nonzero(np.array(endresult) == np.array(list(TRUE_LABELS.lower()[:len(endresult)]))) / len(endresult)
    print ("Accuracy:", acc * 100)

    amp.stop()


In [8]:
data_m = loadmat(TRAIN_DATA)
target_chars = data_m.get('TargetChar')[0][60:]
TRUE_LABELS = target_chars

In [9]:
TRUE_LABELS

'W_EKTLBWXEPOUIKZERYOOTHQI'

# 1. Training (off-line)

In [22]:
clf = train(TRAIN_DATA)

# 2. Online experiment

In [40]:
bbuffer = BlockBuffer(12)
cnt = load_bci_TESTdata(TRAIN_DATA)

In [54]:
cnt.names

['time', 'channel']

In [83]:
amp = RReeplayAmp()
amp.configure(data=cnt.data, marker=cnt.markers, channels=cnt.axes[-1], fs=cnt.fs,
              realtime=False, blocksize_samples=12)

In [ ]:
online_experiment(amp, clf)

DEBUG:__main__:Markers processed: 
DEBUG:__main__:0
DEBUG:__main__:Markers processed: 
DEBUG:__main__:1
DEBUG:__main__:Markers processed: 
DEBUG:__main__:2
DEBUG:__main__:Markers processed: 
DEBUG:__main__:3
DEBUG:__main__:Markers processed: 
DEBUG:__main__:4
DEBUG:__main__:Markers processed: 
DEBUG:__main__:5
DEBUG:__main__:Markers processed: 
DEBUG:__main__:6
DEBUG:__main__:Markers processed: 
DEBUG:__main__:7
DEBUG:__main__:Markers processed: 
DEBUG:__main__:8
DEBUG:__main__:Markers processed: 
DEBUG:__main__:9
DEBUG:__main__:Markers processed: 
DEBUG:__main__:10


P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
time 7.439136505126953
P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
time 8.471250534057617
P300 classification metrics...
recall:  0.0 	precision:  0.0 	f1:  0.0 	accuracy:  1.0
time 8.936405181884766
P300 classification metrics...
recall:  1.0 	precision:  1.0 	f1:  1.0 	accuracy:  1.0
time 8.433341979980469
P300 classification metrics...
recall:  1.0 	precision:  1.0 	f1:  1.0 	accuracy:  1.0
time 11.440515518188477
P300 classification metrics...
recall:  1.0 	precision:  1.0 	f1:  1.0 	accuracy:  1.0
time 9.920358657836914
P300 classification metrics...
recall:  0.5 	precision:  1.0 	f1:  0.6666666666666666 	accuracy:  0.8571428571428571
time 7.802486419677734
P300 classification metrics...
recall:  0.5 	precision:  1.0 	f1:  0.6666666666666666 	accuracy:  0.875
time 9.920597076416016
P300 classification metrics...
recall:  0.5 	precision:  1.0 	f1:  0.666

DEBUG:__main__:Markers processed: 
DEBUG:__main__:11
DEBUG:__main__:Markers processed: 
DEBUG:__main__:12
DEBUG:__main__:Markers processed: 
DEBUG:__main__:13
DEBUG:__main__:Markers processed: 
DEBUG:__main__:14
DEBUG:__main__:Markers processed: 
DEBUG:__main__:15
DEBUG:__main__:Markers processed: 
DEBUG:__main__:16
DEBUG:__main__:Markers processed: 
DEBUG:__main__:17


P300 classification metrics...
recall:  0.5 	precision:  0.5 	f1:  0.5 	accuracy:  0.8181818181818182
time 16.65019989013672
P300 classification metrics...
recall:  0.6666666666666666 	precision:  0.6666666666666666 	f1:  0.6666666666666666 	accuracy:  0.8333333333333334
time 10.91146469116211
P300 classification metrics...
recall:  0.6666666666666666 	precision:  0.6666666666666666 	f1:  0.6666666666666666 	accuracy:  0.8461538461538461
time 11.904716491699219
P300 classification metrics...
recall:  0.6666666666666666 	precision:  0.6666666666666666 	f1:  0.6666666666666666 	accuracy:  0.8571428571428571
time 8.928298950195312
P300 classification metrics...
recall:  0.75 	precision:  0.75 	f1:  0.75 	accuracy:  0.8666666666666667
time 20.33543586730957
P300 classification metrics...
recall:  0.75 	precision:  0.75 	f1:  0.75 	accuracy:  0.875
time 9.423971176147461
P300 classification metrics...
recall:  0.75 	precision:  0.6 	f1:  0.6666666666666665 	accuracy:  0.8235294117647058
tim

DEBUG:__main__:Markers processed: 
DEBUG:__main__:18
DEBUG:__main__:Markers processed: 
DEBUG:__main__:19
DEBUG:__main__:Markers processed: 
DEBUG:__main__:20
DEBUG:__main__:Markers processed: 
DEBUG:__main__:21
DEBUG:__main__:Markers processed: 
DEBUG:__main__:22
DEBUG:__main__:Markers processed: 
DEBUG:__main__:23
DEBUG:__main__:Markers processed: 
DEBUG:__main__:24


...

DEBUG:__main__:Markers processed: 
DEBUG:__main__:59
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouij
DEBUG:__main__:Markers processed: 
DEBUG:__main__:60
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouij
DEBUG:__main__:Markers processed: 
DEBUG:__main__:61
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouij
DEBUG:__main__:Markers processed: 
DEBUG:__main__:62
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouij
DEBUG:__main__:Markers processed: 
DEBUG:__main__:63
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouij


Character prediction metrics...
Current accuracy: 86.66666666666667
time 22.322893142700195
P300 classification metrics...
recall:  0.5695652173913044 	precision:  0.3564625850340136 	f1:  0.4384937238493724 	accuracy:  0.7568840579710145
Character prediction metrics...
Current accuracy: 86.66666666666667
time 22.31597900390625
P300 classification metrics...
recall:  0.5695652173913044 	precision:  0.3564625850340136 	f1:  0.4384937238493724 	accuracy:  0.7569721115537849
Character prediction metrics...
Current accuracy: 86.66666666666667
time 18.361330032348633
P300 classification metrics...
recall:  0.5695652173913044 	precision:  0.3564625850340136 	f1:  0.4384937238493724 	accuracy:  0.7570601013758146
Character prediction metrics...
Current accuracy: 86.66666666666667
time 16.422033309936523
P300 classification metrics...
recall:  0.5695652173913044 	precision:  0.3564625850340136 	f1:  0.4384937238493724 	accuracy:  0.7571480275063337
Character prediction metrics...
Current accur

DEBUG:__main__:Markers processed: 
DEBUG:__main__:64
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouij
DEBUG:__main__:Markers processed: 
DEBUG:__main__:65
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouij
DEBUG:__main__:Markers processed: 
DEBUG:__main__:66
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouij
DEBUG:__main__:Markers processed: 
DEBUG:__main__:67
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouij
DEBUG:__main__:Markers processed: 
DEBUG:__main__:68
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouij


P300 classification metrics...
recall:  0.5695652173913044 	precision:  0.35597826086956524 	f1:  0.4381270903010034 	accuracy:  0.7569620253164557
Character prediction metrics...
Current accuracy: 86.66666666666667
time 14.879941940307617
P300 classification metrics...
recall:  0.5683297180043384 	precision:  0.35597826086956524 	f1:  0.4377610693400167 	accuracy:  0.7566883586406363
Character prediction metrics...
Current accuracy: 86.66666666666667
time 25.783538818359375
P300 classification metrics...
recall:  0.5670995670995671 	precision:  0.35597826086956524 	f1:  0.4373956594323874 	accuracy:  0.7564148897723166
Character prediction metrics...
Current accuracy: 86.66666666666667
time 17.35973358154297
P300 classification metrics...
recall:  0.5670995670995671 	precision:  0.35597826086956524 	f1:  0.4373956594323874 	accuracy:  0.7565028901734104
Character prediction metrics...
Current accuracy: 86.66666666666667
time 18.35155487060547
P300 classification metrics...
recall:  0.

DEBUG:__main__:Markers processed: 
DEBUG:__main__:69
DEBUG:__main__:W_EKTLBWXEPOUIKZERYOOTHQI
DEBUG:__main__:w_iktlbwxepouij
DEBUG:__main__:Markers processed: 
DEBUG:__main__:70


In [13]:


from __future__ import division

import time

import numpy as np

from libmushu.amplifier import Amplifier


class RReeplayAmp(Amplifier):

    def configure(self, data, marker, channels, fs, realtime=True, blocksize_ms=None, blocksize_samples=None):
        """

        Parameters
        ----------
        data
        marker
        channels
        fs
        realtime
        blocksize_ms : float
            blocksize in milliseconds
        blocksize_samples : int
            blocksize in samples

        Raises
        ------
        TypeError :
            if ``blocksize_ms`` and ``blocksize_s`` are given.

        """
        if [blocksize_ms, blocksize_samples].count(None) != 1:
            raise TypeError("blocksize_ms and blocksize_samples are mutually exclusive.")

        self.data = data
        # slow python
        self.marker = marker
        # fast numpy
        self.marker_ts = np.array([ts for ts, s in marker])
        self.marker_s = np.array([s for ts, s in marker])
        self.channels = channels
        self.fs = fs
        self.realtime = realtime

        if blocksize_ms:
            samples = fs * (blocksize_ms / 1000)
            if not samples.is_integer():
                raise ValueError("Resulting blocksize is not integer, please fix the blocksize_ms.")
            self.samples = samples
        if blocksize_samples:
            self.samples = blocksize_samples

    def start(self):
        self.last_sample_time = time.time()
        self.pos = 0

    def stop(self):
        pass

    def get_data(self):
        """

        Returns
        -------
        chunk, markers: Markers is time in ms since relative to the
        first sample of that block.

        """
        if self.realtime:
            elapsed = time.time() - self.last_sample_time
            blocks = (self.fs * elapsed) // self.samples
            samples = int(blocks * self.samples)
            
        else:
            samples = self.samples
      
        elapsed = samples / self.fs
        self.last_sample_time += elapsed
        # data
        chunk = self.data[self.pos:int(self.pos+samples)]
        #self.data = self.data[samples:]
        # markers

        # slow python version
        #markers = [x for x in self.marker if x[0] < elapsed * 1000]
        #self.marker = [x for x in self.marker if x[0] >= elapsed * 1000]
        #self.marker = [[x[0] - elapsed * 1000, x[1]] for x in self.marker]

        # fast numpy version
        mask = self.marker_ts < (elapsed * 1000) 
        markers = list(zip(self.marker_ts[mask], self.marker_s[mask]))
        self.marker_ts = self.marker_ts[~mask]
        self.marker_s = self.marker_s[~mask]
        self.marker_ts -= elapsed * 1000

        self.pos += samples
        return chunk, markers

    def get_channels(self):
        return self.channels

    def get_sampling_frequency(self):
        return self.fs

    @staticmethod
    def is_available():
        return True
